### Методика обновления тарифов в RoamDB
Данная методика применялась для обновления тарифов Partner

In [6]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import sys
from importlib import reload
from os.path import join, normpath
import os
import re
import pandas as pd
import pyodbc

tariff_list = 'Partner Mobile Costs 2019'
downloads = 'C:/Users/balob/Downloads/DATA'
file_new_tariffs = 'Partner_1_6_2019.csv'

In [16]:
#Загрузить данные с новыми тарифами с полями 'DataIncrement_KB', 'OutgoingVoiceIncrement_SEC'

df_tar_updates=pd.read_csv(join(downloads, file_new_tariffs), sep=';')
df_tar_updates.head()

,TADIG,MCCMNC,Partner Name,Country,Operator,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB
0,ALBVF,27602,Vodafone AL,Albania,VF,0.03,0,0.0062,0.003,1,1
1,CZECM,23003,Vodafone CZ,Czech Republic,VF,0.03,0,0.0062,0.003,1,1
2,DEUD2,26202,Vodafone DE,Germany,VF,0.03,0,0.0062,0.003,1,1
3,GHAGT,62002,Vodafone GH,Ghana,VF,0.03,0,0.0062,0.003,1,1
4,GRCPF,20205,Vodafone GR,Greece,VF,0.03,0,0.0062,0.003,1,1


In [17]:
#Загружаем данные из RoamDB для обновляемых TADIG

#Connect to RoamDB
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.19.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')

sql_srt=\
'''
SELECT tl.name AS LIST_NAME, op.Name AS OPERATOR,tc.value AS TADIG,tr.*
FROM Operators op, TadigCodes tc, tariffs tr, tarifflists tl
WHERE tc.id = op.RoamingTadigCodeID
AND tr.OperatorID = op.id
AND tl.Name = ''' + "'" + str(tariff_list) + "'" + '''
AND tl.ID = tr.TariffListID
'''

df_cur_tar = pd.read_sql_query(sql_srt, cnxn)
df_cur_tar.IsDiscounted = df_cur_tar.IsDiscounted.astype(int)

sql_srt=\
'''
select t.Value TADIG,t.OperatorID from TadigCodes t
'''

df_tadig_book = pd.read_sql_query(sql_srt, cnxn)

cnxn.close()

display(df_cur_tar.head())
display(df_tadig_book.head())

,LIST_NAME,OPERATOR,TADIG,ID,OperatorID,TariffListID,IncomingVoice,OutgoingVoice,Text,Data,DataIncrement_KB,OutgoingVoiceIncrement_SEC,IsDiscounted,Support4G,Blocked
0,Partner Mobile Costs 2019,Vodafone,ALBVF,4551,12,28,0.0,0.0439,0.0062,0.0030,1.0,1.0,1,True,False
1,Partner Mobile Costs 2019,Andorra Telecom S A U,ANDMA,4552,17,28,0.0,0.2000,0.0300,0.0500,1.0,60.0,1,False,False
2,Partner Mobile Costs 2019,Cable and Wireless,AIACW,4553,20,28,0.0,0.1606,0.0128,0.0257,1.0,60.0,1,False,False
3,Partner Mobile Costs 2019,C & W,ATGCW,4554,23,28,0.0,0.1500,0.0120,0.0240,1.0,60.0,1,False,False
4,Partner Mobile Costs 2019,Movistar/Telefonica,ARGTM,4555,27,28,0.0,0.0728,0.0240,0.0180,1.0,1.0,1,False,False


,TADIG,OperatorID
0,AFGAW,4
1,AFGAR,5
2,AFG55,6
3,AFGTD,7
4,ALBAM,9


In [18]:
# NEW !!! Объединяем новые тарифы (df_tar_updates) с таблицей TADID из RoamDB (df_tadig_book)
# Таким образом добавляем к таблице новых тарифов OperatorID.
# Иначе, если объединять по TADIG, то для неосновных TADIG будут дубли полей в результате с пустыми значениями

df_tar_updates_OperatorID = pd.merge(df_tar_updates,df_tadig_book,how='left',on='TADIG')
df_tar_updates_OperatorID.head()

,TADIG,MCCMNC,Partner Name,Country,Operator,MO_CALL_RATE,MT_CALL_RATE,MO_SMS_RATE,DATA_RATE,OutgoingVoiceIncrement_SEC,DataIncrement_KB,OperatorID
0,ALBVF,27602,Vodafone AL,Albania,VF,0.03,0,0.0062,0.003,1,1,12
1,CZECM,23003,Vodafone CZ,Czech Republic,VF,0.03,0,0.0062,0.003,1,1,238
2,DEUD2,26202,Vodafone DE,Germany,VF,0.03,0,0.0062,0.003,1,1,338
3,GHAGT,62002,Vodafone GH,Ghana,VF,0.03,0,0.0062,0.003,1,1,343
4,GRCPF,20205,Vodafone GR,Greece,VF,0.03,0,0.0062,0.003,1,1,354


In [24]:
# NEW !!! Объединяем таблицу обновлений (df_tar_updates_OperatorID) с текущими тарифами (df_cur_tar)
# Меняем старые значения на новые
# Выгружаем в csv

df=pd.merge(df_tar_updates_OperatorID, df_cur_tar,how='left',on='OperatorID')

df_res=DataFrame()
df_res[['TADIG','ID','OperatorID','TariffListID',\
        'IncomingVoice','OutgoingVoice','Text','Data','DataIncrement_KB',\
        'OutgoingVoiceIncrement_SEC','IsDiscounted','Support4G','Blocked',]]\
= df[['TADIG_x','ID','OperatorID','TariffListID',\
        'MT_CALL_RATE','MO_CALL_RATE','MO_SMS_RATE','DATA_RATE','DataIncrement_KB_x',\
        'OutgoingVoiceIncrement_SEC_x','IsDiscounted','Support4G','Blocked',]].copy()

df_res.to_csv(join(downloads, 'NEW_TARIFFS_result.csv'),index=False)
df_res.head()

,TADIG,ID,OperatorID,TariffListID,IncomingVoice,OutgoingVoice,Text,Data,DataIncrement_KB,OutgoingVoiceIncrement_SEC,IsDiscounted,Support4G,Blocked
0,ALBVF,4551,12,28,0,0.03,0.0062,0.003,1,1,1,True,False
1,CZECM,4594,238,28,0,0.03,0.0062,0.003,1,1,1,True,False
2,DEUD2,4612,338,28,0,0.03,0.0062,0.003,1,1,1,True,False
3,GHAGT,4614,343,28,0,0.03,0.0062,0.003,1,1,1,False,False
4,GRCPF,4617,354,28,0,0.03,0.0062,0.003,1,1,1,True,False
